In [2]:
import pandas as pd
import soundfile
import librosa
import numpy as np
import re, os
from tqdm import tqdm
import warnings

In [3]:
from pipeline.preprocessing import feature_opensmile

## Feature Extraction

In [4]:
def feature_chromagram(waveform, sample_rate):
    # STFT computed here explicitly; mel spectrogram and MFCC functions do this under the hood
    stft_spectrogram=np.abs(librosa.stft(waveform))
    # Produce the chromagram for all STFT frames and get the mean of each column of the resulting matrix to create a feature array
    chromagram=np.mean(librosa.feature.chroma_stft(S=stft_spectrogram, sr=sample_rate).T,axis=0)
    return chromagram

def feature_melspectrogram(waveform, sample_rate, n_mels=128):
    # Produce the mel spectrogram for all STFT frames and get the mean of each column of the resulting matrix to create a feature array
    # Using 8khz as upper frequency bound should be enough for most speech classification tasks
    melspectrogram=np.mean(librosa.feature.melspectrogram(y=waveform, sr=sample_rate, n_mels=n_mels, fmax=8000).T,axis=0)
    return melspectrogram

def feature_mfcc(waveform, sample_rate, n_mfcc=28):
    # Compute the MFCCs for all STFT frames and get the mean of each column of the resulting matrix to create a feature array
    # 40 filterbanks = 40 coefficients
    mfc_coefficients=np.mean(librosa.feature.mfcc(y=waveform, sr=sample_rate, n_mfcc=n_mfcc).T, axis=0) 
    return mfc_coefficients

In [5]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import librosa
import numpy as np
from scipy import signal
import opensmile
import audiofile
import os
import math

def feature_opensmile(
    waveform: np.ndarray,
    sample_rate: int = 4000,
    one_d: bool = False,
    feature_set=opensmile.FeatureSet.eGeMAPSv02,
    short: bool = False,
) -> pd.DataFrame:
    """
    Calculate the opensmile features for each 2 second. Step = 1 sec. there are 25 features in total.
    If the input waveform is x seconds, then the output DataFrame will be x rowx * 25 columns.
    Ref for eGeMAPSv02: https://sail.usc.edu/publications/files/eyben-preprinttaffc-2015.pdf
    """
    smile = opensmile.Smile(
        feature_set=feature_set,
        feature_level=(
            opensmile.FeatureLevel.LowLevelDescriptors
            if one_d is False else
            opensmile.FeatureLevel.Functionals
        )
    )
    features_df = smile.process_signal(waveform, sample_rate)
    if not short:
        return features_df if one_d is False else features_df.to_numpy()[0]
    else:
        return features_df[
            [  # use this
                "F3frequency_sma3nz",
                "F3bandwidth_sma3nz",
                "F2frequency_sma3nz",
                "F2bandwidth_sma3nz",
                "F1frequency_sma3nz",
                "F1bandwidth_sma3nz",
                "F3amplitudeLogRelF0_sma3nz",
                "F1amplitudeLogRelF0_sma3nz",
                "F2amplitudeLogRelF0_sma3nz",
                "logRelF0-H1-A3_sma3nz",
            ]
        ]

In [6]:
n_chroma = 12
n_mels = 128
n_mfcc = 84

def get_features(file):
    # load an individual soundfile
     with soundfile.SoundFile(file) as audio:
        waveform = audio.read(dtype="float32")
        sample_rate = audio.samplerate # 4000
        # compute features of soundfile
        chromagram = feature_chromagram(waveform, sample_rate)
        melspectrogram = feature_melspectrogram(waveform, sample_rate, n_mels)
        mfc_coefficients = feature_mfcc(waveform, sample_rate, n_mfcc)
        opensmile_fts = feature_opensmile(waveform, sample_rate, one_d=True)
        
        feature_matrix=np.array([])
        # use np.hstack to stack our feature arrays horizontally to create a feature matrix
        feature_matrix = np.hstack((
            chromagram,
            melspectrogram,
            mfc_coefficients,
            opensmile_fts
            ))
        
        return feature_matrix

In [7]:
dataset_info = pd.read_csv('assets/the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data.csv')

outcome_mapping = {'Normal': 1, 'Abnormal': 0}
dataset_info['Mapped_Outcome'] = dataset_info['Outcome'].map(outcome_mapping)
y_dict = dict(zip(dataset_info['Patient ID'], dataset_info['Mapped_Outcome']))

print(y_dict)

{2530: 0, 9979: 0, 9983: 0, 13918: 0, 14241: 0, 14998: 0, 23625: 0, 24160: 0, 29045: 0, 29378: 0, 31737: 0, 33151: 0, 36327: 0, 38337: 0, 39043: 0, 39403: 0, 39456: 0, 40058: 0, 40798: 0, 40840: 0, 43852: 1, 44514: 0, 45843: 0, 46065: 0, 46532: 1, 46579: 0, 46778: 0, 47002: 0, 49558: 0, 49561: 0, 49562: 0, 49568: 1, 49572: 1, 49574: 0, 49577: 1, 49585: 0, 49595: 0, 49598: 1, 49607: 0, 49610: 0, 49618: 0, 49622: 0, 49627: 0, 49628: 0, 49630: 0, 49631: 1, 49638: 0, 49641: 0, 49653: 1, 49659: 0, 49661: 1, 49669: 0, 49678: 1, 49683: 0, 49687: 0, 49691: 0, 49704: 0, 49712: 0, 49719: 1, 49729: 0, 49735: 0, 49745: 0, 49748: 0, 49751: 0, 49754: 0, 49761: 0, 49776: 0, 49808: 1, 49821: 0, 49823: 0, 49824: 0, 49829: 0, 49832: 1, 49838: 0, 49839: 1, 49842: 0, 49850: 0, 49853: 1, 49854: 0, 49873: 0, 49876: 0, 49896: 1, 49897: 1, 49900: 0, 49930: 1, 49931: 1, 49946: 0, 49952: 1, 49959: 1, 49960: 1, 49963: 0, 49966: 0, 49968: 1, 49969: 1, 49970: 1, 49974: 1, 49978: 0, 49979: 1, 49980: 0, 49983: 1, 49

In [8]:
def filter_files_by_keywords_and_extension(folder_path, keywords, extension):
    filtered_files = []
    for filename in os.listdir(folder_path):
        if any(keyword in filename for keyword in keywords) and filename.endswith(extension):
            filtered_files.append(filename)
    return filtered_files

folder_path = 'assets/the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data'
keywords = ['TV','AV','PV','MV']
extension = '.wav'

filtered_files = filter_files_by_keywords_and_extension(folder_path, keywords, extension)
filtered_files[:5]


['13918_AV.wav',
 '13918_MV.wav',
 '13918_PV.wav',
 '13918_TV.wav',
 '14241_AV.wav']

In [9]:
def load_data(filtered_files):
    X, y = [], []
    count = 0
    for file in tqdm(filtered_files):
        file_path = os.path.join(folder_path, file)
        features = get_features(file_path)
        file_number = int(re.match(r'^([^_]*)', file)[1])
        label = y_dict[file_number]
        X.append(features)
        y.append(label)
        count += 1
        # print('\r' + f'Processed {count}/{len(filtered_files)} audio samples', end=' ')
    print()  # Print a newline after the loop completes
    return np.array(X), np.array(y)

warnings.filterwarnings('ignore')
features, labels = load_data(filtered_files)

100%|██████████| 3159/3159 [1:03:57<00:00,  1.21s/it]   

In [37]:
pd.DataFrame(features).to_csv("./assets/feature.csv", header=False, index=False)
# features2 = np.array(pd.read_csv("./assets/feature.csv", header=None))
# features2.shape == features.shape

In [11]:
print('How many samples in total: ', len(labels))

print('How many samples are Normal: ', sum(labels))

How many samples in total:  3159
How many samples are Normal:  1632


In [12]:
print(f'\nAudio samples represented: {features.shape[0]}')
print(f'Numerical features extracted per sample: {features.shape[1]}')
features_df = pd.DataFrame(features) # make it pretty for display
features_df



print('n_chroma', n_chroma)
print('n_mels',n_mels)
print('mfcc', n_mfcc)



Audio samples represented: 3159
Numerical features extracted per sample: 312
n_chroma 12
n_mels 128
mfcc 84


## Feature Processing

In [13]:
# We would usually use df.describe(), but it provides a bit of a mess of information we don't need at the moment.
def print_features(df):
    # Check chromagram feature values
    features_df_chromagram = df.loc[:,:11]
    chroma_min = features_df_chromagram.min().min()
    chroma_max = features_df_chromagram.max().max()
    # stack all features into a single series so we don't get a mean of means or stdev of stdevs
    chroma_mean = features_df_chromagram.stack().mean()
    chroma_stdev = features_df_chromagram.stack().std()
    print(f'{n_chroma} Chromagram features:       \
    min = {chroma_min:.3f}, \
    max = {chroma_max:.3f}, \
    mean = {chroma_mean:.3f}, \
    deviation = {chroma_stdev:.3f}') 

    # Check mel spectrogram feature values
    features_df_melspectrogram = df.loc[:,n_chroma:n_chroma+n_mels-1]
    mel_min = features_df_melspectrogram.min().min()
    mel_max = features_df_melspectrogram.max().max()
    # stack all features into a single series so we don't get a mean of means or stdev of stdevs
    mel_mean = features_df_melspectrogram.stack().mean()
    mel_stdev = features_df_melspectrogram.stack().std()
    print(f'\n{n_mels} Mel Spectrogram features: \
    min = {mel_min:.3f}, \
    max = {mel_max:.3f}, \
    mean = {mel_mean:.3f}, \
    deviation = {mel_stdev:.3f}')

    # Check MFCC feature values
    features_df_mfcc = df.loc[:,n_chroma+n_mels:n_chroma+n_mels+n_mfcc-1]
    mfcc_min = features_df_mfcc.min().min()
    mfcc_max = features_df_mfcc.max().max()
    # stack all features into a single series so we don't get a mean of means or stdev of stdevs
    mfcc_mean = features_df_mfcc.stack().mean()
    mfcc_stdev = features_df_mfcc.stack().std()
    print(f'\n{n_mfcc} MFCC features:             \
    min = {mfcc_min:.3f},\
    max = {mfcc_max:.3f},\
    mean = {mfcc_mean:.3f},\
    deviation = {mfcc_stdev:.3f}')
    
print_features(features_df)

12 Chromagram features:           min = 0.164,     max = 0.997,     mean = 0.808,     deviation = 0.076

128 Mel Spectrogram features:     min = 0.000,     max = 4547.607,     mean = 1.039,     deviation = 12.969

84 MFCC features:                 min = -344.715,    max = 197.198,    mean = -1.062,    deviation = 26.660


In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = StandardScaler()
# keep our unscaled features just in case we need to process them alternatively
features_scaled = features 
features_scaled = scaler.fit_transform(features_scaled)

scaler = MinMaxScaler()
# keep our unscaled features just in case we need to process them alternatively
features_minmax = features
features_minmax = scaler.fit_transform(features_minmax)

In [15]:
print('\033[1m'+'Standard Scaling:\n'+'\033[0m')
features_scaled_df = pd.DataFrame(features_scaled)
print_features(features_scaled_df)

print('\n\n\033[1m'+'MinMax Scaling:\n'+'\033[0m')
features_minmax_df = pd.DataFrame(features_minmax)
print_features(features_minmax_df)

Standard Scaling:

12 Chromagram features:           min = -12.385,     max = 3.475,     mean = 0.000,     deviation = 1.000

128 Mel Spectrogram features:     min = -0.536,     max = 54.209,     mean = -0.000,     deviation = 0.750

84 MFCC features:                 min = -9.718,    max = 10.532,    mean = -0.000,    deviation = 1.000


MinMax Scaling:

12 Chromagram features:           min = 0.000,     max = 1.000,     mean = 0.784,     deviation = 0.105

128 Mel Spectrogram features:     min = 0.000,     max = 1.000,     mean = 0.004,     deviation = 0.026

84 MFCC features:                 min = 0.000,    max = 1.000,    mean = 0.491,    deviation = 0.141


In [16]:
from sklearn.model_selection import train_test_split

############ Unscaled test/train set #############
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    labels, 
    test_size=0.2, 
    random_state=69
)

############ Standard Scaled test/train set ###########
# The labels/classes (y_train, y_test) never change, keep old values 
X_train_scaled, X_test_scaled, _, _ = train_test_split(
    features_scaled, 
    labels, 
    test_size=0.2, 
    random_state=69
)

############# MinMax Scaled test/train set ###############
# The labels/classes (y_train, y_test) never change, keep old values 
X_train_minmax, X_test_minmax, _, _ = train_test_split(
    features_minmax, 
    labels, 
    test_size=0.2, 
    random_state=69
)

## ML Training

In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classification_models = {
    "knn": KNeighborsClassifier(),#(3),
    "svc": SVC(kernel='linear'),#, C=0.025),
    "svc-rbf": SVC(kernel='rbf'),
    "decision tree": DecisionTreeClassifier(),#max_depth=5),
    "random forest": RandomForestClassifier(),#max_depth=5, n_estimators=10, max_features=1),
    "ada boost": AdaBoostClassifier(),
    "GaussianNB": GaussianNB(),
    "QDA": QuadraticDiscriminantAnalysis()
}



In [18]:
from sklearn.metrics import roc_auc_score, f1_score

scores = []
for model_name in classification_models:
    print(f"training model: {model_name}")
    model = classification_models[model_name]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    auc = roc_auc_score(y_test, y_pred)  # Calculate AUC
    f1 = f1_score(y_test, y_pred)  # Calculate F1 score
    score = model.score(X_test, y_test)
    model_name = type(model).__name__
    if model_name == 'SVC' and model.kernel == 'rbf':
        model_name += ' RBF kernel'
    print(f'{100*score:.2f}% auc={auc} f1={f1}')
    scores.append((model_name, f'{100*score:.2f}%', auc, f1))  # Add AUC and F1 score to scores list
# Make it pretty
scores_df = pd.DataFrame(scores, columns=['Classifier', 'Accuracy Score', 'AUC', 'F1 Score'])
scores_df.sort_values(by='Accuracy Score', axis=0, ascending=False)


training model: knn
training model: svc
training model: svc-rbf
training model: decision tree
training model: random forest
training model: ada boost
training model: GaussianNB
training model: QDA


,Classifier,Accuracy Score,AUC,F1 Score
4,RandomForestClassifier,58.39%,0.575074,0.637241
5,AdaBoostClassifier,58.39%,0.583866,0.605697
1,SVC,57.75%,0.568687,0.631724
3,DecisionTreeClassifier,57.44%,0.573679,0.599106
0,KNeighborsClassifier,55.22%,0.549204,0.586861
6,GaussianNB,55.06%,0.512288,0.690632
2,SVC RBF kernel,53.80%,0.515583,0.640394
7,QuadraticDiscriminantAnalysis,45.57%,0.502587,0.017143


In [3]:
from joblib import dump, load

if not os.path.exists("models"):
    os.mkdir("models")
    
for model_name in classification_models:
    print(f"saving model: {model_name}")
    model = classification_models[model_name]
    dump(model, f'./models/{model_name}.joblib')
    # classification_models[model_name] = load(f'./models/{model_name}.joblib')

In [ ]:
scores = []
for model_name in classification_models:
    print(f"training model: {model_name}")
    model = classification_models[model_name]
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    model_name = type(model).__name__
    if model_name=='SVC' and model.kernel=='rbf': model_name+=' RBF kernel'
    scores.append((model_name,(f'{100*score:.2f}%')))
# Make it pretty
scores_df = pd.DataFrame(scores,columns=['Classifier','Accuracy Score'])
scores_df.sort_values(by='Accuracy Score',axis=0,ascending=False)

In [ ]:
scores = []
for model_name in classification_models:
    print(f"training model: {model_name}")
    model = classification_models[model_name]
    model.fit(X_train_scaled, y_train)
    score = model.score(X_test_scaled, y_test)
    model_name = type(model).__name__
    if model_name=='SVC' and model.kernel=='rbf': model_name+=' RBF kernel'
    scores.append((model_name,(f'{100*score:.2f}%')))
# Make it pretty
scores_df = pd.DataFrame(scores,columns=['Classifier','Accuracy Score'])
scores_df.sort_values(by='Accuracy Score',axis=0,ascending=False)

/Users/ziaoyou/WorkSpace/CodeSpace/Projects/HeartDiseasePrediction/.venv/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Users/ziaoyou/WorkSpace/CodeSpace/Projects/HeartDiseasePrediction/.venv/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


,Classifier,Accuracy Score
2,SVC RBF kernel,66.14%
4,RandomForestClassifier,65.03%
1,SVC,62.34%
5,AdaBoostClassifier,59.02%
0,KNeighborsClassifier,56.49%
7,QuadraticDiscriminantAnalysis,55.85%
6,GaussianNB,54.75%
3,DecisionTreeClassifier,49.53%


In [ ]:
scores = []
for model in classification_models:
    model.fit(X_train_minmax, y_train)
    score = model.score(X_test_minmax, y_test)
    model_name = type(model).__name__
    if model_name=='SVC' and model.kernel=='rbf': model_name+=' RBF kernel'
    scores.append((model_name,(f'{100*score:.2f}%')))
# Make it pretty
scores_df = pd.DataFrame(scores,columns=['Classifier','Accuracy Score'])
scores_df.sort_values(by='Accuracy Score',axis=0,ascending=False)

/Users/ziaoyou/WorkSpace/CodeSpace/Projects/HeartDiseasePrediction/.venv/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/Users/ziaoyou/WorkSpace/CodeSpace/Projects/HeartDiseasePrediction/.venv/lib/python3.11/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


,Classifier,Accuracy Score
1,SVC,63.77%
2,SVC RBF kernel,61.23%
4,RandomForestClassifier,60.76%
7,QuadraticDiscriminantAnalysis,59.49%
5,AdaBoostClassifier,59.02%
0,KNeighborsClassifier,56.01%
6,GaussianNB,54.75%
3,DecisionTreeClassifier,50.32%


## Tuning Models

In [ ]:
from sklearn.svm import SVC

model = SVC(
    C=10,
    gamma='auto',
    kernel='rbf',
    random_state=69
)

model.fit(X_train, y_train)

print(f'SVC Model\'s accuracy on training set is {100*model.score(X_train, y_train):.2f}%')
print(f'SVC Model\'s accuracy on test set is {100*model.score(X_test, y_test):.2f}%')

SVC Model's accuracy on training set is 100.00%
SVC Model's accuracy on test set is 56.96%


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

####### Default kNN  ########
model = KNeighborsClassifier(
)

model.fit(X_train, y_train)

print(f'Default kNN Model\'s accuracy on training set is {100*model.score(X_train, y_train):.2f}%')
print(f'Default kNN Model\'s accuracy on test set is {100*model.score(X_test, y_test):.2f}%\n')

##### (hastily) tuned kNN ######
model = KNeighborsClassifier(
    n_neighbors = 5,
    weights = 'distance',
    algorithm = 'brute',
    leaf_size = 30,
    n_jobs=5
)

model.fit(X_train, y_train)

print(f'kNN Model\'s accuracy on training set is {100*model.score(X_train, y_train):.2f}%')
print(f'kNN Model\'s accuracy on test set is {100*model.score(X_test, y_test):.2f}%')

Default kNN Model's accuracy on training set is 73.13%
Default kNN Model's accuracy on test set is 63.13%

kNN Model's accuracy on training set is 100.00%
kNN Model's accuracy on test set is 63.29%


In [ ]:
from sklearn.ensemble import RandomForestClassifier

####### Default Random Forest ########
model = RandomForestClassifier(
    random_state=69
)

model.fit(X_train, y_train)

print(f'Default Random Forest Model\'s accuracy on training set is {100*model.score(X_train, y_train):.2f}%')
print(f'Default Random Forest Model\'s accuracy on test set is {100*model.score(X_test, y_test):.2f}%\n')


########## Tuned Random Forest #######
model = RandomForestClassifier(
    n_estimators = 500, 
    criterion ='entropy',
    warm_start = True,
    max_features = 'sqrt',
    oob_score = True, # more on this below
    random_state=69  
) 

model.fit(X_train, y_train)

print(f'Random Forest Model\'s accuracy on training set is {100*model.score(X_train, y_train):.2f}%')
print(f'Random Forest Model\'s accuracy on test set is {100*model.score(X_test, y_test):.2f}%')

Default Random Forest Model's accuracy on training set is 100.00%
Default Random Forest Model's accuracy on test set is 63.45%

Random Forest Model's accuracy on training set is 100.00%
Random Forest Model's accuracy on test set is 64.08%
